In [1]:
#import libraries
%gui wx
import sys
import os

#####################
# Import of utils.py functions
#####################
# Required to get utils.py and access its functions
notebook_dir = os.path.abspath("")
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(parent_dir)
sys.path.append('.')
from utils import loadFSL, FSLeyesServer, mkdir_no_exist, interactive_MCQ

####################
# DIPY_HOME should be set prior to import of dipy to make sure all downloads point to the right folder
####################
os.environ["DIPY_HOME"] = "/data/data"


#############################
# Loading fsl and freesurfer within Neurodesk
# You can find the list of available other modules by clicking on the "Softwares" tab on the left
#############################
import lmod
await lmod.purge(force=True)
await lmod.load('fsl/6.0.7.4')
await lmod.load('freesurfer/7.4.1')
await lmod.list()

####################
# Setup FSL path
####################
loadFSL()

###################
# Load all relevant libraries for the lab
##################
import fsl.wrappers
from fsl.wrappers import fslmaths

import mne_nirs
import nilearn
from nilearn.datasets import fetch_development_fmri

import mne
import mne_nirs
import dipy
from dipy.data import fetch_bundles_2_subjects, read_bundles_2_subjects
import xml.etree.ElementTree as ET
import os.path as op
import nibabel as nib
import glob

!pip install antspyx
import ants

import openneuro
from mne.datasets import sample
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report


# Useful imports to define the direct download function below
import requests
import urllib.request
from tqdm import tqdm


# FSL function wrappers which we will call from python directly
from fsl.wrappers import fast, bet
from fsl.wrappers.misc import fslroi
from fsl.wrappers import flirt

# General purpose imports to handle paths, files etc
import glob
import pandas as pd
import numpy as np
import json

Gtk-Message: 14:19:48.362: Failed to load module "canberra-gtk-module"


In [2]:
#import data
bids_root = "/data/data" 
print(os.path.exists(bids_root))
derivatives = op.join(os.path.abspath(""),bids_root, "derivatives/preprocessed_data/subject101410") 
mkdir_no_exist(derivatives)
LR_Motor = bids_root + '/subject101410/fMRI/tfMRI_MOTOR_LR/tfMRI_MOTOR_LR.nii' 
RL_Motor = bids_root + '/subject101410/fMRI/tfMRI_MOTOR_RL/tfMRI_MOTOR_RL.nii' 
print_dir_tree(bids_root, max_depth=5) 

True
|data/
|--- derivatives/
|------ preprocessed_data/
|--------- subject101410/
|--- subject101410/
|------ dataset_description.md
|------ task-motor_bold.json
|------ T1w/
|--------- T1w.nii.gz
|------ fMRI/
|--------- tfMRI_MOTOR_LR/
|------------ events_LR.csv
|------------ tfMRI_MOTOR_LR.nii
|--------- tfMRI_MOTOR_RL/
|------------ events_RL.csv
|------------ tfMRI_MOTOR_RL.nii


In [3]:
#functional preprocessing
import nibabel as nib
import numpy as np

def load_img_data(path):
    img = nib.load(path)
    data = img.get_fdata(dtype=np.float32)  # shape = (X, Y, Z, T)
    return img, data

def brain_mask_from_nonzero(data, threshold=1e-6):
    """
    Create a brain mask based on voxels with mean absolute intensity > threshold.
    The threshold is tiny, so it excludes background zeros but keeps all real voxels.
    """
    mean_signal = np.mean(np.abs(data), axis=3)
    mask = mean_signal > threshold
    return mask

def global_variance_over_mask(data, mask):
    """Compute global variance across all brain voxels and timepoints."""
    brain_ts = data[mask]  # shape = (n_voxels, T)
    return np.var(brain_ts)

# ---- File paths ----
run1 = LR_Motor
run2 = RL_Motor

# ---- Load both runs ----
img1, d1 = load_img_data(run1)
img2, d2 = load_img_data(run2)

print(img1.shape)
print(img2.shape)

(91, 109, 91, 284)
(91, 109, 91, 284)


In [4]:
# ---- Make masks (using small threshold) ----
mask1 = brain_mask_from_nonzero(d1, threshold=1e-6)
mask2 = brain_mask_from_nonzero(d2, threshold=1e-6)

# ---- Compute global variance for each run ----
var1 = global_variance_over_mask(d1, mask1)
var2 = global_variance_over_mask(d2, mask2)

# ---- Rescale so variance = 1 ----
d1_scaled = d1 / np.sqrt(var1)
d2_scaled = d2 / np.sqrt(var2)

print(global_variance_over_mask(d1_scaled, mask1))
print(global_variance_over_mask(d2_scaled, mask2))


Gtk-Message: 13:49:24.963: Failed to load module "canberra-gtk-module"


0.99999756
1.0000005


In [5]:
# ---- Concatenate along time (4th dimension) ----
concat = np.concatenate((d1_scaled, d2_scaled), axis=3)

print(concat.shape)

(91, 109, 91, 568)


In [6]:
# ---- Save output ----
save_path = "/home/jovyan/Data/group_project"
out_path = save_path +"/fMRI_motor_concat_var1.nii.gz"
mkdir_no_exist(save_path)

out_img = nib.Nifti1Image(concat, img1.affine, img1.header)
nib.save(out_img, out_path)

print(f"✅ Saved concatenated, variance-normalized fMRI:\n{out_path}")

✅ Saved concatenated, variance-normalized fMRI:
/home/jovyan/Data/group_project/fMRI_motor_concat_var1.nii.gz


In [7]:
#fsleyesDisplay = FSLeyesServer() 
#fsleyesDisplay.show() 
#fsleyesDisplay.resetOverlays() 
#fsleyesDisplay.load(out_path)

In [11]:
from fsl.wrappers import mcflirt 
save_path = "/home/jovyan/Data/group_project"
out_path = save_path+"/fMRI_motor_concat_var1.nii.gz" 
moco_path = save_path+"/moco_mot_corr"
mkdir_no_exist(moco_path)
mcflirt(infile=out_path,o=moco_path+'/moco', plots=True, report=True, dof=6, mats=True)

Processed data will be saved as /home/jovyan/Data/group_project/moco_mot_corr/moco

McFLIRT v 2.0 - FMRI motion correction

Reading time series... 
first iteration - 8mm scaling, set tolerance
Rescaling reference volume [284] to 8 mm pixels
Registering volumes ... [285][286][287][288][289][290][291][292][293][294][295][296][297][298][299][300][301][302][303][304][305][306][307][308][309][310][311][312][313][314][315][316][317][318][319][320][321][322][323][324][325][326][327][328][329][330][331][332][333][334][335][336][337][338][339][340][341][342][343][344][345][346][347][348][349][350][351][352][353][354][355][356][357][358][359][360][361][362][363][364][365][366][367][368][369][370][371][372][373][374][375][376][377][378][379][380][381][382][383][384][385][386][387][388][389][390][391][392][393][394][395][396][397][398][399][400][401][402][403][404][405][406][407][408][409][410][411][412][413][414][415][416][417][418][419][420][421][422][423][424][425][426][427][428][429][430][431]

refnum = 284
Original_refvol = -1


Registering volumes ... [285][286][287][288][289][290][291][292][293][294][295][296][297][298][299][300][301][302][303][304][305][306][307][308][309][310][311][312][313][314][315][316][317][318][319][320][321][322][323][324][325][326][327][328][329][330][331][332][333][334][335][336][337][338][339][340][341][342][343][344][345][346][347][348][349][350][351][352][353][354][355][356][357][358][359][360][361][362][363][364][365][366][367][368][369][370][371][372][373][374][375][376][377][378][379][380][381][382][383][384][385][386][387][388][389][390][391][392][393][394][395][396][397][398][399][400][401][402][403][404][405][406][407][408][409][410][411][412][413][414][415][416][417][418][419][420][421][422][423][424][425][426][427][428][429][430][431][432][433][434][435][436][437][438][439][440][441][442][443][444][445][446][447][448][449][450][451][452][453][454][455][456][457][458][459][460][461][462][463][464][465][466][467][468][469][470][471][472][473][474][475][476][477][478][479][

{}

In [12]:
save_path = "/home/jovyan/Data/group_project"
out_path = save_path+"/fMRI_motor_concat_var1.nii.gz" 
moco_path = save_path+"/moco_mot_corr"
fsleyesDisplay = FSLeyesServer() 
fsleyesDisplay.show() 
fsleyesDisplay.resetOverlays() 
fsleyesDisplay.load(moco_path+"/moco.nii.gz")


(ipykernel_launcher.py:7727): Gdk-WARNING **: 14:33:53.219: gdkdrawable-x11.c:952 drawable is not a pixmap or window

(ipykernel_launcher.py:7727): GLib-GObject-CRITICAL **: 14:34:23.039: invalid (NULL) pointer instance

(ipykernel_launcher.py:7727): GLib-GObject-CRITICAL **: 14:34:23.039: g_signal_handler_disconnect: assertion 'G_TYPE_CHECK_INSTANCE (instance)' failed

(ipykernel_launcher.py:7727): Gtk-CRITICAL **: 14:34:23.039: gtk_window_set_modal: assertion 'GTK_IS_WINDOW (window)' failed


In [ ]:
#coregistration

In [ ]:
#Gaussian Smoothing